In [25]:
# Imports
import gym
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt

from dataclasses import dataclass
from typing import List, Dict
import numpy as np
from abc import ABC, abstractmethod

from edinburgh_challenge.constants import police_stations
from edinburgh_challenge.simulation import SimulationWithMaxUtilisation
from edinburgh_challenge.models import NaiveModel

In [26]:
source = "./data.xlsx"

In [27]:
data = pd.read_excel(source)
data["Time"] = (data["Day"]-1)*24 + data["Hour"]
data.columns = [x.lower() for x in data.columns]

## Simulation Environment

## Models

In [14]:
shift_distribution = {
    'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5},
    'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
    'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]
num_officers_per_station = [4, 5, 6]
simulation = SimulationWithMaxUtilisation(data, ps_coords, shift_distribution, 
                        verbose=1)

## Reinforcement Learning

In [28]:
import gym
from gym import spaces

class SimulationEnv(gym.Env):
    """
    A gym wrapper for the RLEnvironmentModified class to make it compatible with gym.
    """
    def __init__(self, simulation):
        
        
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(len(self.env.action_space))
        # Example for using image as input:
        self.observation_space = spaces.Box(low=0, high=255, shape=(HEIGHT, WIDTH, CHANNELS), dtype=np.uint8)

    def step(self, action):
        return self.env.step(action)

    def reset(self):
        return self.env.reset()

    def render(self, mode='human'):
        pass  # You can add rendering logic here if needed

    def close(self):
        pass  # Implement any cleanup necessary

# Usage example:
# simulation_instance = SimulationWithMaxUtilisation(...)
# gym_env = RLGymWrapper(simulation_instance)


In [16]:
class SimulationWithMaxUtilisation:
    SPEED_MPH = 30  # Speed in miles per hour

    def __init__(self, df: pd.DataFrame, station_coords: List[tuple], shift_distribution: ShiftDistribution, verbose:int=0):
        self.df = df.copy()
        self.station_coords = station_coords
        self.shift_distribution = shift_distribution
        self.current_time = 0
        self.officers = { key: [] for key in shift_distribution["Early"].keys() } # Station_1, Station_2 and Station_3
        self.verbose = verbose

        self.timesteps = [0]
        self.completed_hours = []
        
        self.return_times = [] # array keeps a track of return times of officers
        self.done = False
        # This resolved incidents is made to keep a track of when an incident
        # is allocated to an officer. This is to conduct the evaluating model check
        self.resolved_incidents = [] # This list keeps a track and order of the resolved incidents as they are allocated
        self.cumulative_incidents = []  # Global list to track all incidents
        
        # Making the df compatible with data structures
        df.columns = [x.lower() for x in df.columns]
        

    def print_dashboard(self, allocations):
        print(f"\n--- Day {self.current_time // 24 + 1}, Hour {self.current_time % 24} ---")
        pending_cases = [inc for inc in self.cumulative_incidents if not inc.resolved]
        print(f"Pending Cases: {len(pending_cases)}")
        print("Pending Incident URNs:", [inc.urn for inc in pending_cases])

        inv_allocations = {v: k for k, v in allocations.items()}
        
        for station, officers in self.officers.items():
            print(f"\n{station}:")
            for officer in officers:
                # Find the incident the officer is currently assigned to
                if officer.name in inv_allocations.keys():
                    status = inv_allocations[officer.name]
                else: 
                    status = "Busy"
                print(f"  - {officer.name}: {status}")
        
    def calculate_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees)
        """
        # Convert decimal degrees to radians 
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        # Haversine formula 
        dlat = lat2 - lat1 
        dlon = lon2 - lon1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 3956  # Radius of Earth in miles. Use 6371 for kilometers
        return c * r

    def generate_incidents_for_hour(self, global_time):
        hour_incidents = self.df[(self.df['time'] == global_time)]
        incidents = []
        for _, row in hour_incidents.iterrows():
            deployment_time = row.pop("deployment time (hrs)")
            row.pop("time")  # Not needed as we use global_time
            distances = {f'Station_{i+1}': self.calculate_distance(row['latitude'], row['longitude'], lat, lon) 
                         for i, (lat, lon) in enumerate(self.station_coords)}
            incidents.append(Incident(**row, 
                                      deployment_time=deployment_time, 
                                      distances=distances,
                                      global_time=global_time))
        return incidents


    def update_officers_for_shift(self, shift):
        for station, num_officers in self.shift_distribution[shift].items():
            # Update the number of officers for each station
            self.officers[station] = [Officer(f"Officer_{station}_{shift}_{i}", station=station) for i in range(num_officers)]

    def process_allocations(self, allocations):
        if self.verbose > 0:
            print(f"{allocations=}")
        for urn, officer_id in allocations.items():
            incident = next((inc for inc in self.cumulative_incidents if inc.urn == urn), None)
            #print("incident:", incident)
            if officer_id is None:
                if incident:
                    incident.resolved = False
                continue

            officer = next((off for station_officers in self.officers.values() for off in station_officers if off.name == officer_id), None)
            if incident and officer:
                travel_time = incident.distances[officer.station] / self.SPEED_MPH
                officer.available = False  # Mark officer as busy
                officer.return_time = self.current_time + travel_time + incident.deployment_time

                self.timesteps.append(officer.return_time)
                self.return_times.append(officer.return_time)
            
                incident.resolved = True  # Mark incident as resolved
                incident.resolving_officer = officer.name  # Assign officer to incident
                incident.response_time = self.current_time + travel_time # Global time when the response reached
                incident.resolution_time = officer.return_time # Global Time when the incident was resolved
                self.resolved_incidents.append(incident)
                

    def set_next_time(self):
        self.timesteps = self.timesteps.sort()
        self.timesteps = 
    
    def update_officer_availability(self):
        for station, officers in self.officers.items():
            for officer in officers:
                if not officer.available and self.current_time >= officer.return_time:
                    officer.available = True

    
    def get_return_times_for_next_hour(self):
        global_time = self.current_time
        return_times_for_next_hour = [rt for rt in self.return_times if rt <= global_time + 1]
        return sorted(return_times_for_next_hour)
                    
                    
    def step(self, model):
        
        if self.done:
            return {} # For a week-long simulation
        
        day = self.current_time // 24 + 1
        hour = self.current_time % 24
        
        # Update officer availability at the start of each timestep
        self.update_officer_availability()

        # Update officers for shift change
        if hour in [0, 8, 16]:
            shift = 'Early' if hour == 0 else 'Day' if hour == 8 else 'Night'
            #print(f"{shift=}")
            self.update_officers_for_shift(shift)
        
        total_officers = len(self.officers["Station_1"]) + len(self.officers["Station_2"]) + len(self.officers["Station_3"]) 
            
        # Generate and add new incidents
        new_incidents = self.generate_incidents_for_hour(self.current_time)
        self.cumulative_incidents.extend(new_incidents)
        
        # Filter to get only pending incidents
        pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
        allocations = model.make_allocation(pending_incidents, self.officers, self.current_time)
        
        # Process allocations and update the state
        self.process_allocations(allocations)

        # After making the allocations for the hour
        # get to each return time of the officer
        # and make new allocations
        self.set_next_time()
        

        if self.verbose > 2:
            self.print_dashboard(allocations)
        
        
        # Pending cases after allocation
        pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
        #print(f"{pending_incidents=}")
        
        #self.current_time += 1  # Move to the next hour
        self.hour_index += 1
        self.current_time = self.hours[self.hour_index]
        
        if self.verbose > 3:
            input("Press Enter to continue to the next hour...\n")
        #print()
        #print([inc.urn for inc in self.cumulative_incidents])
        #print("*"*50)
        done = self.current_time >= 24 * 7

    # Checks and Analysis
    def analyze_simulation_results(self):

        simulation = self
        
        # Initialize counters and accumulators
        incident_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        resolved_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        within_threshold_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        response_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        resolution_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        deployment_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        thresholds = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}
        total_officer_hours = {}
        unresolved_incidents = 0
    
        # Analyze each incident
        for incident in simulation.cumulative_incidents:
            priority = incident.priority
            incident_counts[priority] += 1
    
            if incident.resolved:
                resolved_counts[priority] += 1
                
                response_time = incident.response_time
                resolution_time = incident.resolution_time
                
                response_times[priority].append(response_time)
                resolution_times[priority].append(resolution_time)
                
                deployment_time = incident.deployment_time
                deployment_times[priority].append(deployment_time)
                
                if incident.resolving_officer not in total_officer_hours.keys():
                    total_officer_hours[incident.resolving_officer]  = 0
                
                time_spent_on_incident = incident.resolution_time - incident.global_time
                total_officer_hours[incident.resolving_officer] += time_spent_on_incident
                
                # Calculate the time from incident report to response arrival
                time_to_response = incident.response_time - incident.global_time
    
    
                # Check if the response was within the threshold
                if time_to_response <= thresholds[priority]:
                    within_threshold_counts[priority] += 1
            else:
                unresolved_incidents += 1
    
        # Calculate percentages and mean times
        completion_percentages = {p: (resolved_counts[p] / incident_counts[p]) * 100 if incident_counts[p] > 0 else 0 for p in incident_counts}
        mean_response_times = {p: np.mean(response_times[p]) if response_times[p] else 0 for p in response_times}
        mean_deployment_times = {p: np.mean(deployment_times[p]) if deployment_times[p] else 0 for p in deployment_times}
        threshold_compliance = {p: (within_threshold_counts[p] / resolved_counts[p]) * 100 if resolved_counts[p] > 0 else 0 for p in incident_counts}
    
        # Calculate officer utilization
        #for station in simulation.officers.values():
        #    for officer in station:
        #        if not officer.available:
        #            total_officer_hours[officer.name] += (simulation.current_time - officer.return_time)
        
        #officer_utilization = sum(total_officer_hours.values()) / (len(total_officer_hours) * simulation.current_time) * 100
        sum_total_office_hours = sum(total_officer_hours.values())
        unresolved_incident_percentage = (unresolved_incidents / len(simulation.cumulative_incidents)) * 100 if simulation.cumulative_incidents else 0
    
        return {
            "Completion Percentages": completion_percentages,
            "Mean Response Times": mean_response_times,
            "Mean Deployment Times": mean_deployment_times,
            "Threshold Compliance": threshold_compliance,
            #"Officer Utilization": officer_utilization,
            "Total Officer Hours": sum_total_office_hours, 
            "Unresolved Incident Percentage": unresolved_incident_percentage
        }

    def check_simulation(self):
        simulation = self
        # Initialize officer assignments based on all shifts
        officer_assignments = {}
        for shift, stations in shift_distribution.items():
            for station, num_officers in stations.items():
                for i in range(num_officers):
                    officer_name = f"Officer_{station}_{shift}_{i}"
                    officer_assignments[officer_name] = []
        
        incident_response = {'Immediate': {'total': 0, 'within_time': 0}, 
                             'Prompt': {'total': 0, 'within_time': 0}, 
                             'Standard': {'total': 0, 'within_time': 0}}
        
        time_travel_occurred = False
    
        for incident in simulation.resolved_incidents:
            if incident.resolved:
                # Check officer assignments and time traveling
                if incident.resolving_officer:
                    officer_assignments[incident.resolving_officer].append(incident.resolution_time)
                    if len(officer_assignments[incident.resolving_officer]) > 1:
                        if officer_assignments[incident.resolving_officer][-2] > incident.resolution_time:
                            #print(f"{officer_assignments[incident.resolving_officer]=}")
                            #print(f"{incident.resolving_officer=}")
                            time_travel_occurred = True
    
                # Count incidents and check response time
                incident_response[incident.priority]['total'] += 1
                target_time = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}[incident.priority]
                if incident.response_time - incident.global_time <= target_time:
                    incident_response[incident.priority]['within_time'] += 1
    
        # Calculate percentages
        for priority in incident_response:
            total = incident_response[priority]['total']
            if total > 0:
                incident_response[priority]['percentage'] = (incident_response[priority]['within_time'] / total) * 100
            else:
                incident_response[priority]['percentage'] = 0
    
        return officer_assignments, incident_response, time_travel_occurred



SyntaxError: invalid syntax (1852785833.py, line 109)

# Shift allocation

In [5]:
class Model(ABC):
    @abstractmethod
    def make_allocation(self, incidents, officers, current_time):
        pass

@dataclass
class Incident:
    urn: str
    latitude: float
    longitude: float
    day:int
    hour:int
    global_time: float  # Time at which the incident was reported (24*day + hour)
    deployment_time: float  # Time taken to resolve the incident after reaching the location
    priority: str
    distances: Dict[str, float]  # Distances from each station
    resolved: bool = False
    resolving_officer: str = None
    response_time: float = None  # Global time when the response arrives at the scene
    resolution_time: float = None  # Global time when the incident is resolved

@dataclass
class Officer:
    name: str
    station: str
    available: bool = True
    return_time: float = 0.0
        
from typing import Dict, List

# Shift distribution structure
# Example: {'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5}, ...}
ShiftDistribution = Dict[str, Dict[str, int]]

In [74]:
from math import radians, cos, sin, asin, sqrt

class SimulationWithMaxUtilisation:
    SPEED_MPH = 30  # Speed in miles per hour

    def __init__(self, df: pd.DataFrame, station_coords: List[tuple], shift_distribution: ShiftDistribution, verbose:int=0):
        self.df = df.copy()
        self.station_coords = station_coords
        self.shift_distribution = shift_distribution
        self.current_time = 0
        self.officers = { key: [] for key in shift_distribution["Early"].keys() } # Station_1, Station_2 and Station_3
        self.verbose = verbose
        self.hours = [i for i in range(24*7 + 1)]
        self.hour_index = 0
        self.return_times = [] # array keeps a track of return times of officers

        # This resolved incidents is made to keep a track of when an incident
        # is allocated to an officer. This is to conduct the evaluating model check
        self.resolved_incidents = [] # This list keeps a track and order of the resolved incidents as they are allocated
    
        
        # Making the df compatible with data structures
        df.columns = [x.lower() for x in df.columns]
        

    def print_dashboard(self, allocations):
        print(f"\n--- Day {self.current_time // 24 + 1}, Hour {self.current_time % 24} ---")
        pending_cases = [inc for inc in self.cumulative_incidents if not inc.resolved]
        print(f"Pending Cases: {len(pending_cases)}")
        print("Pending Incident URNs:", [inc.urn for inc in pending_cases])

        inv_allocations = {v: k for k, v in allocations.items()}
        
        for station, officers in self.officers.items():
            print(f"\n{station}:")
            for officer in officers:
                # Find the incident the officer is currently assigned to
                if officer.name in inv_allocations.keys():
                    status = inv_allocations[officer.name]
                else: 
                    status = "Busy"
                print(f"  - {officer.name}: {status}")
        
    def calculate_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees)
        """
        # Convert decimal degrees to radians 
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        # Haversine formula 
        dlat = lat2 - lat1 
        dlon = lon2 - lon1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 3956  # Radius of Earth in miles. Use 6371 for kilometers
        return c * r

    def generate_incidents_for_hour(self, global_time):
        hour_incidents = self.df[(self.df['time'] == global_time)]
        incidents = []
        for _, row in hour_incidents.iterrows():
            deployment_time = row.pop("deployment time (hrs)")
            row.pop("time")  # Not needed as we use global_time
            distances = {f'Station_{i+1}': self.calculate_distance(row['latitude'], row['longitude'], lat, lon) 
                         for i, (lat, lon) in enumerate(self.station_coords)}
            incidents.append(Incident(**row, 
                                      deployment_time=deployment_time, 
                                      distances=distances,
                                      global_time=global_time))
        return incidents


    def update_officers_for_shift(self, shift):
        for station, num_officers in self.shift_distribution[shift].items():
            # Update the number of officers for each station
            self.officers[station] = [Officer(f"Officer_{station}_{shift}_{i}", station=station) for i in range(num_officers)]

    def process_allocations(self, allocations):
        if self.verbose > 0:
            print(f"{allocations=}")
        for urn, officer_id in allocations.items():
            incident = next((inc for inc in self.cumulative_incidents if inc.urn == urn), None)
            #print("incident:", incident)
            if officer_id is None:
                if incident:
                    incident.resolved = False
                continue

            officer = next((off for station_officers in self.officers.values() for off in station_officers if off.name == officer_id), None)
            if incident and officer:
                travel_time = incident.distances[officer.station] / self.SPEED_MPH
                officer.available = False  # Mark officer as busy
                officer.return_time = self.current_time + travel_time + incident.deployment_time
                
                self.return_times.append(officer.return_time)
            
                incident.resolved = True  # Mark incident as resolved
                incident.resolving_officer = officer.name  # Assign officer to incident
                incident.response_time = self.current_time + travel_time # Global time when the response reached
                incident.resolution_time = officer.return_time # Global Time when the incident was resolved
                self.resolved_incidents.append(incident)
                

                
    def update_officer_availability(self):
        for station, officers in self.officers.items():
            for officer in officers:
                if not officer.available and self.current_time >= officer.return_time:
                    officer.available = True

    
    def get_return_times_for_next_hour(self):
        global_time = self.current_time
        return_times_for_next_hour = [rt for rt in self.return_times if rt <= global_time + 1]
        return sorted(return_times_for_next_hour)
                    
                    
    def run(self, model):
        self.cumulative_incidents = []  # Global list to track all incidents

        while self.current_time < 24 * 7:  # For a week-long simulation
            
            day = self.current_time // 24 + 1
            hour = self.current_time % 24
            
            # Update officer availability at the start of each timestep
            self.update_officer_availability()

            # Update officers for shift change
            if hour in [0, 8, 16]:
                shift = 'Early' if hour == 0 else 'Day' if hour == 8 else 'Night'
                self.update_officers_for_shift(shift)
            
            total_officers = len(self.officers["Station_1"]) + len(self.officers["Station_2"]) + len(self.officers["Station_3"]) 
            #print(f"{total_officers=}")
                
            # Generate and add new incidents
            new_incidents = self.generate_incidents_for_hour(self.current_time)
            self.cumulative_incidents.extend(new_incidents)
            
            # Filter to get only pending incidents
            allocations = model.make_allocation(pending_incidents, self.officers, self.current_time)
            
            # Process allocations and update the state
            self.process_allocations(allocations)
            
            # After making the allocations for the hour
            # get to each return time of the officer
            # and make new allocations
            return_times_within_hour = self.get_return_times_for_next_hour()
            #print(f"{self.current_time=} {return_times_within_hour=}")
            
            for time in return_times_within_hour:
                self.current_time = time
                
                self.update_officer_availability()
                pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
                allocations = model.make_allocation(pending_incidents, self.officers, self.current_time)
                # Process allocations and update the state
                self.process_allocations(allocations)
                self.return_times.remove(time)
                
                

            if self.verbose > 2:
                self.print_dashboard(allocations)
            
            
            # Pending cases after allocation
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{pending_incidents=}")
            
            #self.current_time += 1  # Move to the next hour
            self.hour_index += 1
            self.current_time = self.hours[self.hour_index]
            
            if self.verbose > 3:
                input("Press Enter to continue to the next hour...\n")
            #print()
            #print([inc.urn for inc in self.cumulative_incidents])
            #print("*"*50)

    # Checks and Analysis
    def analyze_simulation_results(self):

        simulation = self
        
        # Initialize counters and accumulators
        incident_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        resolved_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        within_threshold_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        response_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        resolution_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        deployment_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        thresholds = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}
        total_officer_hours = {}
        unresolved_incidents = 0
    
        # Analyze each incident
        for incident in simulation.cumulative_incidents:
            priority = incident.priority
            incident_counts[priority] += 1
    
            if incident.resolved:
                resolved_counts[priority] += 1
                
                response_time = incident.response_time
                resolution_time = incident.resolution_time
                
                response_times[priority].append(response_time)
                resolution_times[priority].append(resolution_time)
                
                deployment_time = incident.deployment_time
                deployment_times[priority].append(deployment_time)
                
                if incident.resolving_officer not in total_officer_hours.keys():
                    total_officer_hours[incident.resolving_officer]  = 0
                
                time_spent_on_incident = incident.resolution_time - incident.global_time
                total_officer_hours[incident.resolving_officer] += time_spent_on_incident
                
                # Calculate the time from incident report to response arrival
                time_to_response = incident.response_time - incident.global_time
    
    
                # Check if the response was within the threshold
                if time_to_response <= thresholds[priority]:
                    within_threshold_counts[priority] += 1
            else:
                unresolved_incidents += 1
    
        # Calculate percentages and mean times
        completion_percentages = {p: (resolved_counts[p] / incident_counts[p]) * 100 if incident_counts[p] > 0 else 0 for p in incident_counts}
        mean_response_times = {p: np.mean(response_times[p]) if response_times[p] else 0 for p in response_times}
        mean_deployment_times = {p: np.mean(deployment_times[p]) if deployment_times[p] else 0 for p in deployment_times}
        threshold_compliance = {p: (within_threshold_counts[p] / resolved_counts[p]) * 100 if resolved_counts[p] > 0 else 0 for p in incident_counts}
    
        # Calculate officer utilization
        #for station in simulation.officers.values():
        #    for officer in station:
        #        if not officer.available:
        #            total_officer_hours[officer.name] += (simulation.current_time - officer.return_time)
        
        #officer_utilization = sum(total_officer_hours.values()) / (len(total_officer_hours) * simulation.current_time) * 100
        sum_total_office_hours = sum(total_officer_hours.values())
        unresolved_incident_percentage = (unresolved_incidents / len(simulation.cumulative_incidents)) * 100 if simulation.cumulative_incidents else 0
    
        return {
            "Completion Percentages": completion_percentages,
            "Mean Response Times": mean_response_times,
            "Mean Deployment Times": mean_deployment_times,
            "Threshold Compliance": threshold_compliance,
            #"Officer Utilization": officer_utilization,
            "Total Officer Hours": sum_total_office_hours, 
            "Unresolved Incident Percentage": unresolved_incident_percentage
        }

    def check_simulation(self):
        simulation = self
        # Initialize officer assignments based on all shifts
        officer_assignments = {}
        for shift, stations in shift_distribution.items():
            for station, num_officers in stations.items():
                for i in range(num_officers):
                    officer_name = f"Officer_{station}_{shift}_{i}"
                    officer_assignments[officer_name] = []
        
        incident_response = {'Immediate': {'total': 0, 'within_time': 0}, 
                             'Prompt': {'total': 0, 'within_time': 0}, 
                             'Standard': {'total': 0, 'within_time': 0}}
        
        time_travel_occurred = False
    
        for incident in simulation.resolved_incidents:
            if incident.resolved:
                # Check officer assignments and time traveling
                if incident.resolving_officer:
                    officer_assignments[incident.resolving_officer].append(incident.resolution_time)
                    if len(officer_assignments[incident.resolving_officer]) > 1:
                        if officer_assignments[incident.resolving_officer][-2] > incident.resolution_time:
                            #print(f"{officer_assignments[incident.resolving_officer]=}")
                            #print(f"{incident.resolving_officer=}")
                            time_travel_occurred = True
    
                # Count incidents and check response time
                incident_response[incident.priority]['total'] += 1
                target_time = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}[incident.priority]
                if incident.response_time - incident.global_time <= target_time:
                    incident_response[incident.priority]['within_time'] += 1
    
        # Calculate percentages
        for priority in incident_response:
            total = incident_response[priority]['total']
            if total > 0:
                incident_response[priority]['percentage'] = (incident_response[priority]['within_time'] / total) * 100
            else:
                incident_response[priority]['percentage'] = 0
    
        return officer_assignments, incident_response, time_travel_occurred

In [6]:
from math import radians, cos, sin, asin, sqrt

class SimulationWithMaxUtilisation:
    SPEED_MPH = 30  # Speed in miles per hour

    def __init__(self, df: pd.DataFrame, station_coords: List[tuple], shift_distribution: ShiftDistribution, verbose:int=0):
        self.df = df.copy()
        self.station_coords = station_coords
        self.shift_distribution = shift_distribution
        self.current_time = 0
        self.officers = { key: [] for key in shift_distribution["Early"].keys() } # Station_1, Station_2 and Station_3
        self.verbose = verbose
        self.hours = [i for i in range(24*7 + 1)]
        self.hour_index = 0
        self.return_times = [] # array keeps a track of return times of officers

        # This resolved incidents is made to keep a track of when an incident
        # is allocated to an officer. This is to conduct the evaluating model check
        self.resolved_incidents = [] # This list keeps a track and order of the resolved incidents as they are allocated
    
        
        # Making the df compatible with data structures
        df.columns = [x.lower() for x in df.columns]
        

    def print_dashboard(self, allocations):
        print(f"\n--- Day {self.current_time // 24 + 1}, Hour {self.current_time % 24} ---")
        pending_cases = [inc for inc in self.cumulative_incidents if not inc.resolved]
        print(f"Pending Cases: {len(pending_cases)}")
        print("Pending Incident URNs:", [inc.urn for inc in pending_cases])

        inv_allocations = {v: k for k, v in allocations.items()}
        
        for station, officers in self.officers.items():
            print(f"\n{station}:")
            for officer in officers:
                # Find the incident the officer is currently assigned to
                if officer.name in inv_allocations.keys():
                    status = inv_allocations[officer.name]
                else: 
                    status = "Busy"
                print(f"  - {officer.name}: {status}")
        
    def calculate_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees)
        """
        # Convert decimal degrees to radians 
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        # Haversine formula 
        dlat = lat2 - lat1 
        dlon = lon2 - lon1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 3956  # Radius of Earth in miles. Use 6371 for kilometers
        return c * r

    def generate_incidents_for_hour(self, global_time):
        hour_incidents = self.df[(self.df['time'] == global_time)]
        incidents = []
        for _, row in hour_incidents.iterrows():
            deployment_time = row.pop("deployment time (hrs)")
            row.pop("time")  # Not needed as we use global_time
            distances = {f'Station_{i+1}': self.calculate_distance(row['latitude'], row['longitude'], lat, lon) 
                         for i, (lat, lon) in enumerate(self.station_coords)}
            incidents.append(Incident(**row, 
                                      deployment_time=deployment_time, 
                                      distances=distances,
                                      global_time=global_time))
        return incidents


    def update_officers_for_shift(self, shift):
        for station, num_officers in self.shift_distribution[shift].items():
            # Update the number of officers for each station
            self.officers[station] = [Officer(f"Officer_{station}_{shift}_{i}", station=station) for i in range(num_officers)]

    def process_allocations(self, allocations):
        #print(f"{allocations}=")
        for urn, officer_id in allocations.items():
            incident = next((inc for inc in self.cumulative_incidents if inc.urn == urn), None)
            #print("incident:", incident)
            if officer_id is None:
                if incident:
                    incident.resolved = False
                continue

            officer = next((off for station_officers in self.officers.values() for off in station_officers if off.name == officer_id), None)
            if incident and officer:
                travel_time = incident.distances[officer.station] / self.SPEED_MPH
                officer.available = False  # Mark officer as busy
                officer.return_time = self.current_time + travel_time + incident.deployment_time
                
                self.return_times.append(officer.return_time)
            
                incident.resolved = True  # Mark incident as resolved
                incident.resolving_officer = officer.name  # Assign officer to incident
                incident.response_time = self.current_time + travel_time # Global time when the response reached
                incident.resolution_time = officer.return_time # Global Time when the incident was resolved
                self.resolved_incidents.append(incident)
                

                
    def update_officer_availability(self):
        for station, officers in self.officers.items():
            for officer in officers:
                if not officer.available and self.current_time >= officer.return_time:
                    officer.available = True

    
    def get_return_times_for_next_hour(self):
        global_time = self.current_time
        return_times_for_next_hour = [rt for rt in self.return_times if rt <= global_time + 1]
        return sorted(return_times_for_next_hour)
                    
                    
    def run(self, model):
        self.cumulative_incidents = []  # Global list to track all incidents

        while self.current_time < 24 * 7:  # For a week-long simulation
            
            day = self.current_time // 24 + 1
            hour = self.current_time % 24
            
            # Update officer availability at the start of each timestep
            self.update_officer_availability()

            # Update officers for shift change
            if hour in [0, 8, 16]:
                shift = 'Early' if hour == 0 else 'Day' if hour == 8 else 'Night'
                #print(f"{shift=}")
                self.update_officers_for_shift(shift)
            
            total_officers = len(self.officers["Station_1"]) + len(self.officers["Station_2"]) + len(self.officers["Station_3"]) 
            #print(f"{total_officers=}")
                
            # Generate and add new incidents
            new_incidents = self.generate_incidents_for_hour(self.current_time)
            self.cumulative_incidents.extend(new_incidents)
            
            # Filter to get only pending incidents
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{len(pending_incidents)=}")
            #print(f"{pending_incidents=}")
            # Call the model to allocate incidents
            #print(f"{self.officers=}")
            allocations = model.make_allocation(pending_incidents, self.officers, self.current_time)
            
            # Process allocations and update the state
            self.process_allocations(allocations)
            
            # After making the allocations for the hour
            # get to each return time of the officer
            # and make new allocations
            return_times_within_hour = self.get_return_times_for_next_hour()
            #print(f"{self.current_time=} {return_times_within_hour=}")
            
            for time in return_times_within_hour:
                self.current_time = time
                
                self.update_officer_availability()
                pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
                allocations = model.make_allocation(pending_incidents, self.officers, self.current_time)
                # Process allocations and update the state
                self.process_allocations(allocations)
                self.return_times.remove(time)
                
                

            if self.verbose > 2:
                self.print_dashboard(allocations)
            
            
            # Pending cases after allocation
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{pending_incidents=}")
            
            #self.current_time += 1  # Move to the next hour
            self.hour_index += 1
            self.current_time = self.hours[self.hour_index]
            
            if self.verbose > 3:
                input("Press Enter to continue to the next hour...\n")
            #print()
            #print([inc.urn for inc in self.cumulative_incidents])
            #print("*"*50)

    # Checks and Analysis
    def analyze_simulation_results(self):

        simulation = self
        
        # Initialize counters and accumulators
        incident_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        resolved_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        within_threshold_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        response_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        resolution_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        deployment_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        thresholds = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}
        total_officer_hours = {}
        unresolved_incidents = 0
    
        # Analyze each incident
        for incident in simulation.cumulative_incidents:
            priority = incident.priority
            incident_counts[priority] += 1
    
            if incident.resolved:
                resolved_counts[priority] += 1
                
                response_time = incident.response_time
                resolution_time = incident.resolution_time
                
                response_times[priority].append(response_time)
                resolution_times[priority].append(resolution_time)
                
                deployment_time = incident.deployment_time
                deployment_times[priority].append(deployment_time)
                
                if incident.resolving_officer not in total_officer_hours.keys():
                    total_officer_hours[incident.resolving_officer]  = 0
                
                time_spent_on_incident = incident.resolution_time - incident.global_time
                total_officer_hours[incident.resolving_officer] += time_spent_on_incident
                
                # Calculate the time from incident report to response arrival
                time_to_response = incident.response_time - incident.global_time
    
    
                # Check if the response was within the threshold
                if time_to_response <= thresholds[priority]:
                    within_threshold_counts[priority] += 1
            else:
                unresolved_incidents += 1
    
        # Calculate percentages and mean times
        completion_percentages = {p: (resolved_counts[p] / incident_counts[p]) * 100 if incident_counts[p] > 0 else 0 for p in incident_counts}
        mean_response_times = {p: np.mean(response_times[p]) if response_times[p] else 0 for p in response_times}
        mean_deployment_times = {p: np.mean(deployment_times[p]) if deployment_times[p] else 0 for p in deployment_times}
        threshold_compliance = {p: (within_threshold_counts[p] / resolved_counts[p]) * 100 if resolved_counts[p] > 0 else 0 for p in incident_counts}
    
        # Calculate officer utilization
        #for station in simulation.officers.values():
        #    for officer in station:
        #        if not officer.available:
        #            total_officer_hours[officer.name] += (simulation.current_time - officer.return_time)
        
        #officer_utilization = sum(total_officer_hours.values()) / (len(total_officer_hours) * simulation.current_time) * 100
        sum_total_office_hours = sum(total_officer_hours.values())
        unresolved_incident_percentage = (unresolved_incidents / len(simulation.cumulative_incidents)) * 100 if simulation.cumulative_incidents else 0
    
        return {
            "Completion Percentages": completion_percentages,
            "Mean Response Times": mean_response_times,
            "Mean Deployment Times": mean_deployment_times,
            "Threshold Compliance": threshold_compliance,
            #"Officer Utilization": officer_utilization,
            "Total Officer Hours": sum_total_office_hours, 
            "Unresolved Incident Percentage": unresolved_incident_percentage
        }

    def check_simulation(self):
        simulation = self
        # Initialize officer assignments based on all shifts
        officer_assignments = {}
        for shift, stations in shift_distribution.items():
            for station, num_officers in stations.items():
                for i in range(num_officers):
                    officer_name = f"Officer_{station}_{shift}_{i}"
                    officer_assignments[officer_name] = []
        
        incident_response = {'Immediate': {'total': 0, 'within_time': 0}, 
                             'Prompt': {'total': 0, 'within_time': 0}, 
                             'Standard': {'total': 0, 'within_time': 0}}
        
        time_travel_occurred = False
    
        for incident in simulation.resolved_incidents:
            if incident.resolved:
                # Check officer assignments and time traveling
                if incident.resolving_officer:
                    officer_assignments[incident.resolving_officer].append(incident.resolution_time)
                    if len(officer_assignments[incident.resolving_officer]) > 1:
                        if officer_assignments[incident.resolving_officer][-2] > incident.resolution_time:
                            #print(f"{officer_assignments[incident.resolving_officer]=}")
                            #print(f"{incident.resolving_officer=}")
                            time_travel_occurred = True
    
                # Count incidents and check response time
                incident_response[incident.priority]['total'] += 1
                target_time = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}[incident.priority]
                if incident.response_time - incident.global_time <= target_time:
                    incident_response[incident.priority]['within_time'] += 1
    
        # Calculate percentages
        for priority in incident_response:
            total = incident_response[priority]['total']
            if total > 0:
                incident_response[priority]['percentage'] = (incident_response[priority]['within_time'] / total) * 100
            else:
                incident_response[priority]['percentage'] = 0
    
        return officer_assignments, incident_response, time_travel_occurred

In [9]:
naive_model = NaiveModel()

In [23]:
shift_distribution = {
    'Early': {'Station_1': 15, 'Station_2': 0, 'Station_3': 0},
    'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
    'Night': {'Station_1': 40, 'Station_2': 0, 'Station_3': 0}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]

simulation = SimulationWithMaxUtilisation(data, ps_coords, shift_distribution, 
                        verbose=1)

In [24]:
simulation.run(naive_model)
simulation.analyze_simulation_results()

{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 100.0},
 'Mean Response Times': {'Immediate': 88.31999429925446,
  'Prompt': 84.40111515611476,
  'Standard': 87.59917839828113},
 'Mean Deployment Times': {'Immediate': 1.5403225806451613,
  'Prompt': 1.5061196105702366,
  'Standard': 1.4796755725190842},
 'Threshold Compliance': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 100.0},
 'Total Officer Hours': 3641.621307961209,
 'Unresolved Incident Percentage': 0.0}

In [93]:
simulation.run(naive_model)
simulation.analyze_simulation_results()

{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 100.0},
 'Mean Response Times': {'Immediate': 88.31291756055249,
  'Prompt': 84.39669221166272,
  'Standard': 87.59333263354172},
 'Mean Deployment Times': {'Immediate': 1.5403225806451613,
  'Prompt': 1.5061196105702366,
  'Standard': 1.4796755725190842},
 'Threshold Compliance': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 100.0},
 'Total Officer Hours': 3630.004144118147,
 'Unresolved Incident Percentage': 0.0}

In [18]:
import gym
from gym import spaces
import numpy as np

class PoliceAllocationEnv(gym.Env):
    """
    A gym environment for allocating police officers to stations.
    """
    def __init__(self, max_officers, stations=3):
        super(PoliceAllocationEnv, self).__init__()
        self.stations = stations
        self.max_officers = max_officers

        # Define action and observation space
        self.action_space = spaces.MultiDiscrete([max_officers + 1] * stations)
        self.observation_space = spaces.Box(low=0, high=max_officers, shape=(stations,), dtype=np.int32)

        self.state = None

    def reset(self):
        self.state = np.zeros(self.stations, dtype=np.int32)
        return self.state

    def step(self, action):
        # Ensure the total officers do not exceed max_officers
        if sum(action) > self.max_officers:
            reward = -1  # Penalize invalid actions
            done = True
        else:
            self.state = action
            reward = self.calculate_reward()
            done = False  # Can implement a condition to end the episode

        return self.state, reward, done, {}

    def calculate_reward(self):
        # Placeholder: Implement the logic to calculate the reward
        reward = 0
        # Example: Reward could be based on the coverage and efficiency of allocation
        return reward

    def close(self):
        pass  # Optional: Cleanup


In [19]:
import itertools

def generate_shift_distributions(num_officers_per_station, total_shifts=3):
    """
    Generates all possible distributions of officers across three police stations for a given shift.

    :param num_officers_per_station: List of number of officers at each station.
    :param total_shifts: Total number of shifts to consider.
    :return: List of dictionaries representing different officer distributions for each shift.
    """
    distributions = []

    # Generate all combinations of officer distributions for the given numbers
    for distribution in itertools.product(num_officers_per_station, repeat=total_shifts):
        shift_distribution = {
            'Early': {'Station_1': distribution[0], 'Station_2': distribution[1], 'Station_3': distribution[2]},
            'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},  # Keeping day shift constant
            'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}  # Keeping night shift constant
        }
        distributions.append(shift_distribution)

    return distributions


In [36]:
def run_simulations(data, ps_coords, num_officers_per_station, model):
    distributions = generate_shift_distributions(num_officers_per_station)
    simulation_results = []

    for distribution in distributions:
        simulation = SimulationWithMaxUtilisation(data, ps_coords, distribution, verbose=1)
        simulation.run(model)
        res = simulation.analyze_simulation_results()
        result = calculate_composite_metric(res)
        simulation_results.append((distribution, result))
    
    return simulation_results

In [37]:
results = run_simulations(data, ps_coords, [4, 5, 6], naive_model)

In [34]:
def calculate_composite_metric(simulation_results, compliance_weight=2, deployment_weight=1, completion_weight=1):
    """
    Calculate a composite metric from simulation results.

    :param simulation_results: The results from the simulation.
    :param compliance_weight: The weight for threshold compliance in the metric.
    :param deployment_weight: The weight for mean deployment time in the metric.
    :param completion_weight: The weight for completion percentage in the metric.
    :return: The composite metric value.
    """
    mean_deployment_time = sum(simulation_results['Mean Deployment Times'].values()) / len(simulation_results['Mean Deployment Times'])
    threshold_compliance = sum(simulation_results['Threshold Compliance'].values()) / len(simulation_results['Threshold Compliance'])
    completion_percentage = sum(simulation_results['Completion Percentages'].values()) / len(simulation_results['Completion Percentages'])

    # Calculate composite metric
    metric = (compliance_weight * threshold_compliance +
              deployment_weight * mean_deployment_time +
              completion_weight * completion_percentage)

    return metric

In [28]:
naive_model = NaiveModel()

In [38]:
results

[({'Early': {'Station_1': 4, 'Station_2': 4, 'Station_3': 4},
   'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
   'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}},
  301.5087059212448),
 ({'Early': {'Station_1': 4, 'Station_2': 4, 'Station_3': 5},
   'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
   'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}},
  301.5087059212448),
 ({'Early': {'Station_1': 4, 'Station_2': 4, 'Station_3': 6},
   'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
   'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}},
  301.5087059212448),
 ({'Early': {'Station_1': 4, 'Station_2': 5, 'Station_3': 4},
   'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
   'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}},
  301.5087059212448),
 ({'Early': {'Station_1': 4, 'Station_2': 5, 'Station_3': 5},
   'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
   'Night': {'Station_1': 13, 'Station

In [43]:
def generate_early_shift_distributions(total_officers=15, stations=3):
    """
    Generates all possible distributions of officers across three police stations 
    for the early shift that sum up to a specific total.

    :param total_officers: Total number of officers to be distributed.
    :param stations: Number of stations.
    :return: List of tuples representing different officer distributions for the early shift.
    """
    distributions = []

    # Iterate through all possible combinations
    for distribution in itertools.product(range(total_officers + 1), repeat=stations):
        if sum(distribution) == total_officers:
            distributions.append(distribution)

    return distributions

In [48]:

shift_distribution = {
    'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5},
    'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
    'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]

naive_model = NaiveModel()

In [62]:
early_shift_dist = generate_early_shift_distributions()
simulation_results = []
for dist in distributions:
    new_shift_distribution = dict(shift_distribution)
    new_shift_distribution["Early"] = {'Station_1': dist[0], 'Station_2':dist[1], 'Station_3':dist[2]}
    simulation = SimulationWithMaxUtilisation(data, ps_coords, new_shift_distribution, verbose=1)
    simulation.run(naive_model)
    print(simulation.officers)
    res = simulation.analyze_simulation_results()
    result = calculate_composite_metric(res)
    simulation_results.append((new_shift_distribution, result))

{'Station_1': [Officer(name='Officer_Station_1_Night_0', station='Station_1', available=True, return_time=167.37891324748762), Officer(name='Officer_Station_1_Night_1', station='Station_1', available=False, return_time=168.96036832642662), Officer(name='Officer_Station_1_Night_2', station='Station_1', available=True, return_time=162.64887202736614), Officer(name='Officer_Station_1_Night_3', station='Station_1', available=True, return_time=0.0), Officer(name='Officer_Station_1_Night_4', station='Station_1', available=True, return_time=0.0), Officer(name='Officer_Station_1_Night_5', station='Station_1', available=True, return_time=0.0), Officer(name='Officer_Station_1_Night_6', station='Station_1', available=True, return_time=0.0), Officer(name='Officer_Station_1_Night_7', station='Station_1', available=True, return_time=0.0), Officer(name='Officer_Station_1_Night_8', station='Station_1', available=True, return_time=0.0), Officer(name='Officer_Station_1_Night_9', station='Station_1', ava

KeyboardInterrupt: 

In [55]:
simulation_results

[({'Early': {'Station_1': 0, 'Station_2': 0, 'Station_3': 15},
   'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
   'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}},
  301.5087059212448),
 ({'Early': {'Station_1': 0, 'Station_2': 1, 'Station_3': 14},
   'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
   'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}},
  301.5087059212448),
 ({'Early': {'Station_1': 0, 'Station_2': 2, 'Station_3': 13},
   'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
   'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}},
  301.5087059212448),
 ({'Early': {'Station_1': 0, 'Station_2': 3, 'Station_3': 12},
   'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
   'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}},
  301.5087059212448),
 ({'Early': {'Station_1': 0, 'Station_2': 4, 'Station_3': 11},
   'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
   'Night': {'Station_1': 13, 'St

In [66]:
sim = SimulationWithMaxUtilisation(data, ps_coords, {'Early': {'Station_1': 2, 'Station_2': 2, 'Station_3': 11},
 'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
 'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}})

In [65]:
new_shift_distribution

{'Early': {'Station_1': 2, 'Station_2': 2, 'Station_3': 11},
 'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
 'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}}

In [67]:
sim.run(naive_model)

In [68]:
sim.officers

{'Station_1': [Officer(name='Officer_Station_1_Night_0', station='Station_1', available=True, return_time=167.37891324748762),
  Officer(name='Officer_Station_1_Night_1', station='Station_1', available=False, return_time=168.96036832642662),
  Officer(name='Officer_Station_1_Night_2', station='Station_1', available=True, return_time=162.64887202736614),
  Officer(name='Officer_Station_1_Night_3', station='Station_1', available=True, return_time=0.0),
  Officer(name='Officer_Station_1_Night_4', station='Station_1', available=True, return_time=0.0),
  Officer(name='Officer_Station_1_Night_5', station='Station_1', available=True, return_time=0.0),
  Officer(name='Officer_Station_1_Night_6', station='Station_1', available=True, return_time=0.0),
  Officer(name='Officer_Station_1_Night_7', station='Station_1', available=True, return_time=0.0),
  Officer(name='Officer_Station_1_Night_8', station='Station_1', available=True, return_time=0.0),
  Officer(name='Officer_Station_1_Night_9', statio